# Orthophoto Processing with WMS Services

This notebook aims to access the most up-to-date orthophoto (year 2024) of the Basque Country through a WMS (Web Map Service), process the image to match the required spatial resolution, and generate a single RGB raster for the study area.

The workflow is divided into the following main steps:

1. **Dividing the study area into tiles**: The study area is divided into smaller tiles with a specified resolution and size. This step ensures efficient data handling and processing by breaking the larger area into manageable fragments.

2. **Connecting to the WMS and downloading tiles**: A connection to the WMS service is established to retrieve the orthophoto for the specified tiles. Each tile is downloaded as an RGB image in GeoTIFF format, preserving visual quality and geospatial metadata.

3. **Creating the final raster**: The downloaded tiles are merged to produce a single RGB raster that covers the entire extent of the study area. The final raster maintains the desired spatial resolution and is ready for subsequent analysis.

This approach ensures efficient geospatial data processing while maintaining the quality and precision required for geographic analysis.

## 1. Read in the geojson file and split the study area in smaller tiles

In [1]:
# Import libraries
from owslib.wms import WebMapService
import parameters
import geopandas as gpd
import rasterio
from rasterio.crs import CRS
import os
from rasterio.transform import from_bounds
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import box
import math
from rasterio.io import MemoryFile
from rasterio.merge import merge

In [2]:
# Create raster folder (if not created yet)
if not os.path.exists(parameters.output_dir):
    os.makedirs(parameters.output_dir)

In [3]:
# Read the file as a geodataframe
polygon = gpd.read_file(parameters.input_file)

In [4]:
# Bounding boxes of the polygon
minx, miny, maxx, maxy = polygon.total_bounds

bbox = (minx, miny, maxx, maxy)

In the next cell we determine that the spatial resolution must be **2 meters** and that the maximum download size is **16.000 pixels (4.000x4.000)**. These values ​​are used because this resolution may be adequate to perform a change analysis (a common task when working with orthophotos or satellite images, for example) and pixel values ​​greater than 4.000x4.000 can saturate the service.

In [5]:
# Spatial resolution
spatial_resolution = 2

# Maximun width and height
max_size = 4000

Taking into consideration  the spatial resolution and maximun sizes described above, we'll split the study area in smaller tiles.

In [6]:
# Split bbox in smaller tiles
def split_bbox(bbox, resolution, max_size):
    # How many tiles we need with the resolution and sizes provided
    width_total = bbox[2] - bbox[0]
    height_total = bbox[3] - bbox[1]

    num_tiles_x = math.ceil(width_total / (max_size * resolution))
    num_tiles_y = math.ceil(height_total / (max_size * resolution))

    # Bbox for each tile
    tiles = []
    tile_width = width_total / num_tiles_x
    tile_height = height_total / num_tiles_y

    for i in range(num_tiles_x):
        for j in range(num_tiles_y):
            minx = bbox[0] + i * tile_width
            maxx = minx + tile_width
            miny = bbox[1] + j * tile_height
            maxy = miny + tile_height
            tiles.append((minx, miny, maxx, maxy))

    return tiles

In [7]:
# Run the function
tiles = split_bbox(bbox, spatial_resolution, max_size)

# Check the new bounding boxes
tiles

[(551406.3437000001, 4774340.409, 557746.7466000002, 4780465.103),
 (551406.3437000001, 4780465.103, 557746.7466000002, 4786589.797)]

We see that with the parameters provided the polygon has been split in two smaller tiles.

## 2. WMS connection and tiles download

Now it's time to make the WMS connection and download the tiles with the resolution and size defined above.

In [8]:
# WMS connection
wms = WebMapService(parameters.wms_url, version='1.3.0')

In [9]:
# Identifier and title for each layer in the service
for layer_id, layer_info in wms.contents.items():
    print(f"Identifier {layer_id}, Title: {layer_info.title}")

Identifier ORTO_EGUNERATUENA_MAS_ACTUALIZADA, Title: ORTO_EGUNERATUENA_MAS_ACTUALIZADA
Identifier ORTO_2023, Title: ORTO_2023
Identifier ORTO_2023_IrRG, Title: ORTO_2023_IrRG
Identifier ORTO_2022, Title: ORTO_2022
Identifier ORTO_2022_IrRG, Title: ORTO_2022_IrRG
Identifier ORTO_2021_IrRG, Title: ORTO_2021_IrRG
Identifier ORTO_2021, Title: ORTO_2021
Identifier ORTO_2020_IrRG, Title: ORTO_2020_IrRG
Identifier ORTO_2020, Title: ORTO_2020
Identifier ORTO_2019_COSTA, Title: ORTO_2019_COSTA
Identifier ORTO_2019_COSTA_IrRG, Title: ORTO_2019_COSTA_IrRG
Identifier ORTO_2019_IrRG, Title: ORTO_2019_IrRG
Identifier ORTO_2019, Title: ORTO_2019
Identifier ORTO_2018_IrRG, Title: ORTO_2018_IrRG
Identifier ORTO_2018, Title: ORTO_2018
Identifier ORTO_2017_IrRG, Title: ORTO_2017_IrRG
Identifier ORTO_2017, Title: ORTO_2017
Identifier ORTO_URBANA_2017, Title: ORTO_URBANA_2017
Identifier ORTO_2016_IrRG, Title: ORTO_2016_IrRG
Identifier ORTO_2016, Title: ORTO_2016
Identifier ORTO_2015_IrRG, Title: ORTO_2015_

In [10]:
# Making the request for each tile and save the result as a geotiff
def download_tile(bbox, width, height, tile_id):
    # Request
    response = wms.getmap(
        layers=['ORTO_EGUNERATUENA_MAS_ACTUALIZADA'],
        bbox=bbox,
        srs='EPSG:25830',
        size=(width, height),
        format='image/geotiff',
        transparent=True
    )

    # Create tiles folder
    if not os.path.exists(parameters.tiles_dir):
        os.makedirs(parameters.tiles_dir)
    
    # Using MemoryFile and rasterio to create new rasters
    with MemoryFile(response.read()) as memfile:
        with memfile.open() as src:
            # Read the raster information
            data = src.read()

            # Spatial transformation between pixels and the coordinate system
            transform = from_bounds(bbox[0], bbox[1], bbox[2], bbox[3], width, height)
            
            # Output file name
            filename = f'tile_{tile_id}.tif'
            
            # Save the file with all the metadata
            with rasterio.open(
                os.path.join(parameters.tiles_dir, filename), 'w',
                driver='GTiff',
                height=src.height,
                width=src.width,
                count=src.count,
                dtype=src.dtypes[0],
                crs='EPSG:25830',
                transform=transform
            ) as dst:
                dst.write(data)
    
    return os.path.join(parameters.tiles_dir, filename)

In [11]:
# Empty list to store all tiles names
images = []

In [12]:
# Calculate the size for each tile and download
for idx, tile_bbox in enumerate(tiles):
    width_tile = int((tile_bbox[2] - tile_bbox[0]) / spatial_resolution)
    height_tile = int((tile_bbox[3] - tile_bbox[1]) / spatial_resolution)
    image_filename = download_tile(tile_bbox, width_tile, height_tile, idx)
    images.append(image_filename)

c:\Users\User\miniconda3\envs\gis\Lib\site-packages\rasterio\io.py:138: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  return DatasetReader(mempath, driver=driver, sharing=sharing, **kwargs)


## 3. Merging the tiles

If we want to join all the tiles and create a single final file in geotiff format we can use the following code.

In [15]:
# Merge tiles in one raster
def merge_images(images):
    # Read each tile
    src_files_to_mosaic = []
    for img in images:
        src = rasterio.open(img)
        src_files_to_mosaic.append(src)

    # Merge
    mosaic, out_trans = merge(src_files_to_mosaic)

    # Output raster metadata
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans
    })

    # Store as a geotiff moduan gorde
    with rasterio.open(os.path.join(parameters.tiles_dir, 'merged.tif'), 'w', **out_meta) as dest:
        dest.write(mosaic)

In [16]:
# Create the raster
merge_images(images)